In [11]:
import sys
import os

sys.path.append(os.path.join(os.getcwd(), '..', 'toolkits'))

import torch


tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda:3" if torch.cuda.is_available() else "cpu"),
}
SMOKE_TEST = os.environ.get("SMOKE_TEST")

### Problem setup


In [12]:
from design import Design
test_f = Design()
def problem(X):
    return test_f.evaluate(X.to(torch.float64))
bounds = test_f.bounds
d = 4
M = 2

In [13]:
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import normalize, unnormalize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from gpytorch.kernels import RBFKernel, ScaleKernel
def evaluate_slack(X, ref = torch.tensor([-1.9,-2.25]).to(**tkwargs)):
    Y = problem(X)
    vio_raw = Y -ref
    return (vio_raw).sum(dim = -1, keepdim = True)

def generate_initial_data(n):
    # generate training data
    train_x = draw_sobol_samples(bounds=bounds, n=n, q=1).squeeze(1)
    train_obj, train_con = problem(train_x)
    # negative values imply feasibility in botorch
    # train_con = -evaluate_slack(train_x)
    return train_x, train_obj, -train_con

# base = RBFKernel()
# covar_module = ScaleKernel(
# base_kernel=base,
# )
def initialize_model(train_x, train_obj, train_con):
    # define models for objective and constraint
    train_x = normalize(train_x, bounds)
    train_y = torch.cat([train_obj, train_con], dim=-1)
    models = []
    for i in range(train_y.shape[-1]):
        models.append(
            SingleTaskGP(
                train_x, train_y[..., i : i + 1], outcome_transform=Standardize(m=1),train_Yvar= torch.zeros((train_x.shape[0],1)) + 0.05**2)
            )
        
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [14]:
from botorch.acquisition.multi_objective.monte_carlo import (
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.sampling import sample_simplex


BATCH_SIZE = 1
NUM_RESTARTS = 4
RAW_SAMPLES =4

standard_bounds = torch.zeros(2, 4, **tkwargs)
standard_bounds[1] = 1


def optimize_qnehvi_and_get_observation(model, train_x, train_obj, train_con, sampler):
    """Optimizes the qNEHVI acquisition function, and returns a new candidate and observation."""
    train_x = normalize(train_x, bounds)
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point= torch.tensor([-8,-8]),  # use known reference point
        X_baseline=train_x,
        sampler=sampler,
        prune_baseline=False,
        # define an objective that specifies which outcomes are the objectives
        objective=IdentityMCMultiOutputObjective(outcomes=[0, 1]),
        # specify that the constraint is on the last outcome
        constraints=[
                lambda Z: Z[..., -1],  # First constraint
                lambda Z: Z[..., -2],  # Second constraint
                lambda Z: Z[..., -3],  # Third constraint
                lambda Z: Z[..., -4],  # Fourth constraint
            ],
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj, new_con = problem(new_x)  + torch.randn_like(problem(new_x)) * 0.05
    # negative values imply feasibility in botorch
    return new_x, new_obj, -new_con

In [ ]:
import time
import warnings

from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.pareto import is_non_dominated


warnings.filterwarnings("ignore")
random_seeds = [83810, 14592, 3278, 97196, 36048, 32098, 29256, 18289, 96530, 13434, 88696, 97080, 71482, 11395, 77397, 55302, 4165, 3905, 12280, 28657, 30495, 66237, 78907, 3478, 73563,
26062, 93850, 85181, 91924, 71426, 54987, 28893, 58878, 77236, 36463, 851, 99458, 20926, 91506, 55392, 44597, 36421, 20379, 28221, 44118, 13396, 12156, 49797, 12676, 47052]
declared = False
c = 0
N_BATCH = 100
MC_SAMPLES = 128 if not SMOKE_TEST else 16
verbose = True
for seed in random_seeds[:10]:
    torch.manual_seed(seed)
    train_x_qnehvi, train_obj_qnehvi, train_con_qnehvi = generate_initial_data(10)
    train_x_qnehvi = train_x_qnehvi.to(torch.float64)
    hv = Hypervolume(ref_point=torch.tensor([-8,-8]).to(**tkwargs))
    hvs_qnehvi, hvs_random = [], []
    # call helper functions to generate initial training data and initialize model



    train_x_random, train_obj_random, train_con_random = (
        train_x_qnehvi,
        train_obj_qnehvi,
        train_con_qnehvi,
    )

    mll_qnehvi, model_qnehvi = initialize_model(
        train_x_qnehvi, train_obj_qnehvi, train_con_qnehvi
    )

    # compute pareto front
    is_feas = (train_con_qnehvi <= 0).all(dim=-1)
    feas_train_obj = train_obj_qnehvi[is_feas]
    if feas_train_obj.shape[0] > 0:
        pareto_mask = is_non_dominated(feas_train_obj)
        pareto_y = feas_train_obj[pareto_mask]
        # compute hypervolume
        volume = hv.compute(pareto_y)
    else:
        volume = 0.0

    # hvs_qnehvi.append(volume)
    # hvs_random.append(volume)
    # run N_BATCH rounds of BayesOpt after the initial random batch
    for iteration in range(1, N_BATCH + 1):
        t0 = time.monotonic()

        # fit the models
        fit_gpytorch_mll(mll_qnehvi)

        # define the qParEGO and qNEHVI acquisition modules using a QMC sampler
        qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))

        # optimize acquisition functions and get new observations
        new_x_qnehvi, new_obj_qnehvi, new_con_qnehvi = optimize_qnehvi_and_get_observation(
            model_qnehvi, train_x_qnehvi, train_obj_qnehvi, train_con_qnehvi, qnehvi_sampler
        )
        new_x_random, new_obj_random, new_con_random = generate_initial_data(n=BATCH_SIZE)

        # update training points
        train_x_qnehvi = torch.cat([train_x_qnehvi, new_x_qnehvi])
        train_obj_qnehvi = torch.cat([train_obj_qnehvi, new_obj_qnehvi])
        train_con_qnehvi = torch.cat([train_con_qnehvi, new_con_qnehvi])

        train_x_random = torch.cat([train_x_random, new_x_random])
        train_obj_random = torch.cat([train_obj_random, new_obj_random])
        train_con_random = torch.cat([train_con_random, new_con_random])

        # update progress
        for hvs_list, train_obj, train_con in zip(
            (hvs_random, hvs_qnehvi),
            (train_obj_random, train_obj_qnehvi),
            (train_con_random,  train_con_qnehvi),
        ):
            # compute pareto front
            is_feas = (train_con <= 0).all(dim=-1)
            feas_train_obj = train_obj[is_feas]
            if feas_train_obj.shape[0] > 0:
                pareto_mask = is_non_dominated(feas_train_obj)
                pareto_y = feas_train_obj[pareto_mask]
                # compute feasible hypervolume
                volume = hv.compute(pareto_y)
            else:
                volume = 0.0
            hvs_list.append(volume)

        # reinitialize the models so they are ready for fitting on next iteration
        # Note: we find improved performance from not warm starting the model hyperparameters
        # using the hyperparameters from the previous iteration
        mll_qnehvi, model_qnehvi = initialize_model(
            train_x_qnehvi, train_obj_qnehvi, train_con_qnehvi
        )

        t1 = time.monotonic()

        if verbose:
            print(
                f"\nBatch {iteration:>2}: Hypervolume (random, qNEHVI) = "
                f"({hvs_random[-1]:>8f}, {hvs_qnehvi[-1]:>8f}), "
                f"time = {t1-t0:>4.2f}.",
                end="",
            )
        else:
            print(".", end="")
    c+=1
    vio = torch.where(train_con_qnehvi > 0, train_con_qnehvi, torch.zeros_like(train_con_qnehvi)).sum(dim = -1)
    torch.save(torch.tensor(hvs_qnehvi), f'hv_design_ehvi_{c}.pt')
    torch.save(vio, f'vio_design_ehvi_{c}.pt')